# CS486 - Artificial Intelligence
## Lesson 18 - Reinforcement Learning

So far our MDP policies have leveraged a great deal of information: The complete state-space, the transition model, and rewards. Real-world agents don't often have perfect information about their environment. In this lesson, we'll look at how to build an optimal policy by learning from the environment. 

In [ ]:
import helpers
from aima.rl import *
from aima.notebook import psource

## Passive Reinforcement Learning

A **passive reinforcement learning** agent does not dictate actions. It simply follows a policy, perhaps a random policy, and gather observations, or **samples**, to determine the value of a state. There are three passive reinforcement learning techniques: Direct utility estimation, adaptive dynamic programming, and temporal-difference learning. 

### Direct Utility Estimation

Direct utility estimation follows a policy until it reaches a terminal state - we'll call this a single **trial**. At each step, it logs the current state and reward. When it reaches the terminal it estimates the utility for each state for *that* iteration by  summing the rewards from that state to the terminal one. It continues to run trials, calculating the average utility of each state.
 
Let's look at the example from the lecture ([image credit](https://lachdata.com/2018/06/30/grid-world-part-1-iterative-policy-evaluation/)):

![Grid World](images/grid_world_policy.jpg)

In [ ]:
from aima.utils import print_table
from aima.mdp import sequential_decision_environment as grid_world

north = ( 0, 1)
south = ( 0,-1)
east  = ( 1, 0)
west  = (-1, 0)

policy = {
    (0, 2): east,  (1, 2): east,  (2, 2): east,   (3, 2): None,
    (0, 1): north,                (2, 1): north,  (3, 1): None,
    (0, 0): north, (1, 0): west,  (2, 0): west,   (3, 0): west, 
}

agent = PassiveDUEAgent(policy, grid_world)

for i in range(200):
    run_single_trial(agent,grid_world)
    agent.estimate_U()

U = grid_world.to_grid({s: round(v,2) for (s, v) in agent.U.items()})
print_table(U)

In [ ]:
psource(agent.estimate_U)

There are a few problems with this approach:

1. This doesn't scale well. Trials in large state-spaces could take a really long time. 
2. DUE doesn't take advantage of the relationship between states. If we knew which actions were likely to lead to which states, we could estimate values faster. 

### Sample-Base Policy Evaluation / Adaptive Dynamic Programming
 
Adaptive dynamic programming builds a transition model, $T(s,a,s')$. It does so by remembering the number of times an action from state, $s$, led transitioned to another state, $s'$ and divides that by the number of trials that included $s$. 

Now that we have a transision model, we can just use **policy evaluation** to determine the optimal policy. 

In [ ]:
agent = PassiveADPAgent(policy, grid_world)

for i in range(200):
    run_single_trial(agent,grid_world)

U = grid_world.to_grid({s: round(v,2) for (s, v) in agent.U.items()})    
print_table(U)

We can see the booking necessary to produce the transition model needed for policy evaluation: 

In [ ]:
for (t,s,a),n in agent.Ns1_sa.items():
    print("Transitioned from", s, "to", t, n, "times")

The problem here is that we are already following a policy and we can't build an accurate transition function unless we can explore all the actions from every state. Even if we could execute arbitrary actions, it would take a lot of samples to build the estimate and the amount of book keeping is prohibitive. 

### Temporal-Difference Learning
 
Keeping models is expensive. Instead, the temporal-difference model makes use of the expected closeness between the utilities of two consecutive states $s$ and $s'$.

$$U^{\pi}(s) \leftarrow U^{\pi}(s) + \alpha \left( R(s) + \gamma U^{\pi}(s') - U^{\pi}(s) \right)$$

This model implicitly incorporates the transition probabilities by being weighed for each state by the number of times it is achieved from the current state. Over a number of iterations, it converges similarly to the Bellman equations.

The advantage of the TD learning model is its relatively simple computation at each step, rather than having to keep track of various counts.

For $n_s$ states and $n_a$ actions the ADP model would have $n_s \times n_a$ numbers $N_{sa}$ and $n_s^2 \times n_a$ numbers $N_{s'|sa}$ to keep track of. The TD model must only keep track of a utility $U(s)$ for each state.

In [ ]:
# use learning rate given in the footnote of the book on page 837 
agent = PassiveTDAgent(policy, grid_world, alpha = lambda n: 60./(59+n))

for i in range(200):
    run_single_trial(agent, grid_world)
    
U = grid_world.to_grid({s: round(v,2) for (s, v) in agent.U.items()})    
print_table(U)    

## Active Reinforcement Learning

Passive learning is a good way to evaluate a given policy, but they don't really help us choose actions. How does an agent learn an optimal policy from scratch in a way that scales? One method is Q-Learning.

### Q-Learning

Q-Learning maintains a table of Q-states for the MDP. It uses a temporal-difference approach to update Q-values without having to keep a path history like adaptive dynamic programming. It also doesn't indiscriminately propogate values back to the start state. Like value iteration, it takes into account that the agent will act optimally in future states.  

Q-Learning is a model-free way will converge to an optimal policy given enough exploration. It converges according to the following equation:

$$ Q_{k+1}(s,a) \leftarrow \sum_{s'}T(s,a,s')\left[R(s,a,s')+\gamma \max_{a'} Q_{k}(s',a')\right] $$


In [ ]:
agent = QLearningAgent(grid_world, Ne=5, Rplus=2, alpha=lambda n: 60./(59+n))

for i in range(200):
    run_single_trial(agent,grid_world)
    
agent.Q

The utility vector is just the largest Q-value from every Q-state:

In [ ]:
U = defaultdict(lambda: -1000.)

for (state, _), value in agent.Q.items():
    if U[state] < value:
        U[state] = value

U = grid_world.to_grid({s: round(v,2) for (s, v) in U.items()})    
print_table(U) 